In [50]:
import numpy as np
import cv2

import keras
from keras.models import load_model

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import PIL

import math
import time

In [2]:
model = load_model('model.h5')

/home/awis/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [60]:
def load_image_pixels(filename, shape):
    height, width, c = filename.shape
    # load the image with the required size
    image = cv2.resize(filename, shape)
    print("Frame_Feed_To_Model  :",image.shape)
    # scale pixel values to [0, 1]
    image = image.astype('float32')
    image /= 255.0
    # add a dimension so that we have one sample
    image = np.expand_dims(image, 0)
    return image, width, height

class BoundBox:
    def __init__(self, xmin, ymin, xmax, ymax, objness = None, classes = None):
        self.xmin = xmin
        self.ymin = ymin
        self.xmax = xmax
        self.ymax = ymax
        self.objness = objness
        self.classes = classes
        self.label = -1
        self.score = -1
 
    def get_label(self):
        if self.label == -1:
            self.label = np.argmax(self.classes)
 
        return self.label
 
    def get_score(self):
        if self.score == -1:
            self.score = self.classes[self.get_label()]
 
        return self.score
 
def _sigmoid(x):
    return 1. / (1. + np.exp(-x))
 
def decode_netout(netout, anchors, obj_thresh, net_h, net_w):
    grid_h, grid_w = netout.shape[:2]
    nb_box = 3
    netout = netout.reshape((grid_h, grid_w, nb_box, -1))
    nb_class = netout.shape[-1] - 5
    boxes = []
    netout[..., :2]  = _sigmoid(netout[..., :2])
    netout[..., 4:]  = _sigmoid(netout[..., 4:])
    netout[..., 5:]  = netout[..., 4][..., np.newaxis] * netout[..., 5:]
    netout[..., 5:] *= netout[..., 5:] > obj_thresh
 
    for i in range(grid_h*grid_w):
        row = i / grid_w
        col = i % grid_w
        for b in range(nb_box):
            # 4th element is objectness score
            objectness = netout[int(row)][int(col)][b][4]
            if(objectness.all() <= obj_thresh): continue
            # first 4 elements are x, y, w, and h
            x, y, w, h = netout[int(row)][int(col)][b][:4]
            x = (col + x) / grid_w # center position, unit: image width
            y = (row + y) / grid_h # center position, unit: image height
            w = anchors[2 * b + 0] * np.exp(w) / net_w # unit: image width
            h = anchors[2 * b + 1] * np.exp(h) / net_h # unit: image height
            # last elements are class probabilities
            classes = netout[int(row)][col][b][5:]
            box = BoundBox(x-w/2, y-h/2, x+w/2, y+h/2, objectness, classes)
            boxes.append(box)
    return boxes
 
def correct_yolo_boxes(boxes, image_h, image_w, net_h, net_w):
    new_w, new_h = net_w, net_h
    for i in range(len(boxes)):
        x_offset, x_scale = (net_w - new_w)/2./net_w, float(new_w)/net_w
        y_offset, y_scale = (net_h - new_h)/2./net_h, float(new_h)/net_h
        boxes[i].xmin = int((boxes[i].xmin - x_offset) / x_scale * image_w)
        boxes[i].xmax = int((boxes[i].xmax - x_offset) / x_scale * image_w)
        boxes[i].ymin = int((boxes[i].ymin - y_offset) / y_scale * image_h)
        boxes[i].ymax = int((boxes[i].ymax - y_offset) / y_scale * image_h)
 
def _interval_overlap(interval_a, interval_b):
    x1, x2 = interval_a
    x3, x4 = interval_b
    if x3 < x1:
        if x4 < x1:
            return 0
        else:
            return min(x2,x4) - x1
    else:
        if x2 < x3:
             return 0
        else:
            return min(x2,x4) - x3
 
def bbox_iou(box1, box2):
    intersect_w = _interval_overlap([box1.xmin, box1.xmax], [box2.xmin, box2.xmax])
    intersect_h = _interval_overlap([box1.ymin, box1.ymax], [box2.ymin, box2.ymax])
    intersect = intersect_w * intersect_h
    w1, h1 = box1.xmax-box1.xmin, box1.ymax-box1.ymin
    w2, h2 = box2.xmax-box2.xmin, box2.ymax-box2.ymin
    union = w1*h1 + w2*h2 - intersect
    return float(intersect) / union
 
def do_nms(boxes, nms_thresh):
    if len(boxes) > 0:
        nb_class = len(boxes[0].classes)
    else:
        return
    for c in range(nb_class):
        sorted_indices = np.argsort([-box.classes[c] for box in boxes])
        for i in range(len(sorted_indices)):
            index_i = sorted_indices[i]
            if boxes[index_i].classes[c] == 0: continue
            for j in range(i+1, len(sorted_indices)):
                index_j = sorted_indices[j]
                if bbox_iou(boxes[index_i], boxes[index_j]) >= nms_thresh:
                    boxes[index_j].classes[c] = 0
                    
# define the labels
labels = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck",
    "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench",
    "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe",
    "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard",
    "sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard",
    "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana",
    "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake",
    "chair", "sofa", "pottedplant", "bed", "diningtable", "toilet", "tvmonitor", "laptop", "mouse",
    "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator",
    "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"]

def get_boxes(boxes, labels, thresh):
    v_boxes, v_labels, v_scores = list(), list(), list()
    # enumerate all boxes
    for box in boxes:
        # enumerate all possible labels
        for i in range(len(labels)):
            # check if the threshold for this label is high enough
            if box.classes[i] > thresh:
                v_boxes.append(box)
                v_labels.append(labels[i])
                v_scores.append(box.classes[i]*100)
                # don't break, many labels may trigger for one box
    return v_boxes, v_labels, v_scores
 
# draw all results
def draw_boxes(frame, v_boxes, v_labels, v_scores, min_dist):
    cal_point=[]
    for i in range(len(v_boxes)):
        box = v_boxes[i]
        # get coordinates
        y1, x1, y2, x2 = box.ymin, box.xmin, box.ymax, box.xmax
        width, height = x2 - x1, y2 - y1
        mid_x, mid_y = (width//2)+x1, y2
        cal_point.append((mid_x,mid_y))
        frame = cv2.rectangle(frame, (x1,y1), (x2,y2), (255,255,0), 1)
        label = "%s (%.3f) %s" % (v_labels[i], v_scores[i], i)
        t_point_x=x1-14
        t_point_y=y1-3
        frame = cv2.putText(frame, label, (t_point_x,t_point_y), cv2.FONT_HERSHEY_SIMPLEX, 0.35, color=(255,255,255), thickness=1)

    print("\nMid point coordinates :")
    print(cal_point,"\n")
    for i in range(len(cal_point)):
        for j in range(i+1,len(cal_point)):
            d=round(math.sqrt(math.pow(cal_point[j][0]-cal_point[i][0],2)+math.pow(cal_point[j][1]-cal_point[i][1],2)),3)
            print("boxes :", i,":", j,"Distance :", d)
            if(d<=min_dist):
                box_i,box_j = v_boxes[i],v_boxes[j]
                y1i, x1i, y2i, x2i = box_i.ymin, box_i.xmin, box_i.ymax, box_i.xmax
                y1j, x1j, y2j, x2j = box_j.ymin, box_j.xmin, box_j.ymax, box_j.xmax
                frame = cv2.rectangle(frame, (x1i,y1i), (x2i,y2i), (0,0,255), 1)
                frame = cv2.rectangle(frame, (x1j,y1j), (x2j,y2j), (0,0,255), 1)


In [61]:
video = "sample.mp4"
cap = cv2.VideoCapture(video)

# Check if video file is opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

anchors = [[116,90, 156,198, 373,326], [30,61, 62,45, 59,119], [10,13, 16,30, 33,23]]
# define the probability threshold for detected objects
class_threshold = 0.6
boxes = list()
rem=list()

while(cap.isOpened()):
    # record start time
    start=time.time()
    
    ret, frame = cap.read()
    frame = cv2.resize(frame, (960, 540))
    
    print("\nCaptured_frame_shape :",frame.shape)
    input_w, input_h = 416, 416
    image, image_w, image_h = load_image_pixels(frame, (input_w, input_h))
    
    yhat = model.predict(image)
    print([a.shape for a in yhat])
    
    # decode the output of the network
    for i in range(len(yhat)):
        boxes += decode_netout(yhat[i][0], anchors[i], class_threshold, input_h, input_w)
    print("box_list size: ",len(boxes))
    # correct the sizes of the bounding boxes for the shape of the image
    correct_yolo_boxes(boxes, image_h, image_w, input_h, input_w)
    
    # suppress non-maximal boxes
    do_nms(boxes, 0.5)
    
    # get the details of the detected objects
    v_boxes, v_labels, v_scores = get_boxes(boxes, labels, class_threshold)
    
    # finding and clearing off the unwanted objects detected before feeding to draw boxes function
    for i in range(len(v_labels)):
        if(v_labels[i]!="person"):
            rem.append(i)
    v_boxes=[i for i in v_boxes if v_boxes.index(i) not in rem]
    v_labels=[i for i in v_labels if v_labels.index(i) not in rem]
    v_scores=[i for i in v_scores if v_scores.index(i) not in rem]
        
    # draw what we found
    min_dist=50
    draw_boxes(frame, v_boxes, v_labels, v_scores, min_dist)
    
    cv2.imshow('frame',frame)
    
    end=time.time()
    print("\nP_T_P_F :",round(end-start,1))
    
    boxes.clear()
    rem.clear()
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Captured_frame_shape : (540, 960, 3)
Frame_Feed_To_Model  : (416, 416, 3)
[(1, 13, 13, 255), (1, 26, 26, 255), (1, 52, 52, 255)]
box_list size:  10647

Mid point coordinates :
[(749, 114), (915, 195), (378, 214), (159, 244), (464, 297), (833, 446), (151, 542), (825, 552), (670, 26), (587, 41), (505, 66), (690, 72), (456, 130), (840, 155), (415, 220)] 

boxes : 0 : 1 Distance : 184.708
boxes : 0 : 2 Distance : 384.241
boxes : 0 : 3 Distance : 604.152
boxes : 0 : 4 Distance : 338.695
boxes : 0 : 5 Distance : 342.462
boxes : 0 : 6 Distance : 735.383
boxes : 0 : 7 Distance : 444.545
boxes : 0 : 8 Distance : 118.258
boxes : 0 : 9 Distance : 177.688
boxes : 0 : 10 Distance : 248.676
boxes : 0 : 11 Distance : 72.422
boxes : 0 : 12 Distance : 293.437
boxes : 0 : 13 Distance : 99.81
boxes : 0 : 14 Distance : 350.417
boxes : 1 : 2 Distance : 537.336
boxes : 1 : 3 Distance : 757.586
boxes : 1 : 4 Distance : 462.391
boxes : 1 : 5 Distance : 264.055
boxes : 1 : 6 Distance : 839.11
boxes : 1 : 7 Di

[(1, 13, 13, 255), (1, 26, 26, 255), (1, 52, 52, 255)]
box_list size:  10647

Mid point coordinates :
[(752, 111), (385, 215), (422, 217), (162, 242), (152, 245), (453, 327), (832, 449), (137, 540), (382, 548), (671, 26), (586, 41), (508, 66), (695, 72), (457, 128), (839, 152), (919, 178)] 

boxes : 0 : 1 Distance : 381.451
boxes : 0 : 2 Distance : 346.606
boxes : 0 : 3 Distance : 604.368
boxes : 0 : 4 Distance : 614.781
boxes : 0 : 5 Distance : 368.859
boxes : 0 : 6 Distance : 347.338
boxes : 0 : 7 Distance : 749.844
boxes : 0 : 8 Distance : 572.598
boxes : 0 : 9 Distance : 117.414
boxes : 0 : 10 Distance : 180.155
boxes : 0 : 11 Distance : 248.115
boxes : 0 : 12 Distance : 69.065
boxes : 0 : 13 Distance : 295.489
boxes : 0 : 14 Distance : 96.177
boxes : 0 : 15 Distance : 179.939
boxes : 1 : 2 Distance : 37.054
boxes : 1 : 3 Distance : 224.629
boxes : 1 : 4 Distance : 234.923
boxes : 1 : 5 Distance : 131.027
boxes : 1 : 6 Distance : 504.544
boxes : 1 : 7 Distance : 408.814
boxes : 1 :

[(1, 13, 13, 255), (1, 26, 26, 255), (1, 52, 52, 255)]
box_list size:  10647

Mid point coordinates :
[(751, 110), (385, 214), (427, 215), (144, 249), (451, 325), (833, 460), (132, 543), (380, 548), (670, 27), (585, 41), (510, 66), (698, 74), (458, 128), (838, 147)] 

boxes : 0 : 1 Distance : 380.489
boxes : 0 : 2 Distance : 340.589
boxes : 0 : 3 Distance : 622.712
boxes : 0 : 4 Distance : 369.087
boxes : 0 : 5 Distance : 359.477
boxes : 0 : 6 Distance : 755.414
boxes : 0 : 7 Distance : 574.008
boxes : 0 : 8 Distance : 115.974
boxes : 0 : 9 Distance : 179.769
boxes : 0 : 10 Distance : 244.984
boxes : 0 : 11 Distance : 64.07
boxes : 0 : 12 Distance : 293.552
boxes : 0 : 13 Distance : 94.541
boxes : 1 : 2 Distance : 42.012
boxes : 1 : 3 Distance : 243.528
boxes : 1 : 4 Distance : 129.139
boxes : 1 : 5 Distance : 511.097
boxes : 1 : 6 Distance : 415.03
boxes : 1 : 7 Distance : 334.037
boxes : 1 : 8 Distance : 340.872
boxes : 1 : 9 Distance : 264.441
boxes : 1 : 10 Distance : 193.724
boxes

In [47]:
math.sqrt?